In [5]:
!pip install yfinance==0.1.80
!pip install https://github.com/buguroo/pyknow/archive/v1.7.0.tar.gz

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.40
    Uninstalling yfinance-0.2.40:
      Successfully uninstalled yfinance-0.2.40
  Using cached https://github.com/buguroo/pyknow/archive/v1.7.0.tar.gz (1.4 MB)
  Preparing metadata (setup.py) ... done


In [1]:
import collections
import collections.abc

if not hasattr(collections, 'Mapping'):
  collections.Mapping = collections.abc.Mapping

In [2]:
from pyknow import Rule, Fact, Field, KnowledgeEngine, DefFacts, NOT, MATCH, AS
import random

In [3]:
!python

^C


# Classes

## WinTotals

In [4]:
class WinTotals(Fact):
    human = Field(int, default=0)
    computer = Field(int, default=0)
    ties = Field(int, default=0)

## Results

In [5]:
class Results(Fact):
    winner = Field(str, mandatory=True)
    loser = Field(str, mandatory=True)
    why = Field(str, mandatory=True)

## ValidAnswer

In [6]:
class ValidAnswer(Fact):
    answer = Field(str, mandatory=True)
    key = Field(str, mandatory=True)

## Action, HumanChoice, ComputerChoice

In [7]:
class Action(Fact):
    pass


class HumanChoice(Fact):
    pass


class ComputerChoice(Fact):
    pass

## RockPaperScissors

# Play

In [8]:
class RockPaperScissors(KnowledgeEngine):

    def yes_or_no(self, question):
        return input(question).upper().startswith('Y')

    @DefFacts()
    def game_rules(self):
        """Declare game rules and valid input keys for the user."""
        self.valid_answers = dict()

        yield Results(winner='rock', loser='scissors', why='Rock smashes scissors')
        yield Results(winner='paper', loser='rock', why='Paper covers rock')
        yield Results(winner='scissors', loser='paper', why='Scissors cut paper')
        yield ValidAnswer(answer='rock', key='r')
        yield ValidAnswer(answer='paper', key='p')
        yield ValidAnswer(answer='scissors', key='s')



    @Rule()
    def startup(self):
        print("Lets play a game!")
        print("You choose rock, paper, or scissors,")
        print("and I'll do the same.")
        self.declare(WinTotals(human=0, computer=0, ties=0))
        self.declare(Action('get-human-move'))

    @Rule(NOT(Action()),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.key))
    def store_valid_answers(self, answer, key):
        self.valid_answers[key] = answer

    #
    # HUMAN MOVE RULES
    #
    @Rule(Action('get-human-move'))
    def get_human_move(self):
        print('\t\t in get_human_move() --> declaring HumanChoice')
        question = ", ".join(
            "{name} ({key})".format(
                name=a[1].title(), key=a[0].upper())
            for a in self.valid_answers.items()) + '? '
        res = input(question).lower()
        self.declare(HumanChoice(res))

    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          ValidAnswer(answer=MATCH.answer,
                      key=MATCH.choice),
          AS.f2 << Action('get-human-move'))
    def good_human_move(self, f1, f2, answer):
        self.retract(f1) # HumanChoice
        self.retract(f2) # Action('get-human-move')
        self.declare(HumanChoice(answer))
        self.declare(Action('get-computer-move'))

    @Rule(AS.f1 << HumanChoice(MATCH.choice),
          NOT(ValidAnswer(key=MATCH.choice)),
          AS.f2 << Action('get-human-move'))
    def bad_human_move(self, f1, f2, choice):
        print('\t\t in bad_human_move() --> declaring get-human-move')
        print("Sorry %s is not a valid answer" % choice)
        self.retract(f1)
        self.retract(f2)
        self.declare(Action('get-human-move'))

    #
    # COMPUTER MOVE RULES
    #
    @Rule(AS.f1 << Action('get-computer-move'))
    def get_computer_move(self, f1):
        choice = random.choice(list(self.valid_answers.values()))
        print(f'\t\t in get-computer-move computer selected{choice}')
        self.retract(f1) # get-computer-move
        self.declare(ComputerChoice(choice))
        self.declare(Action('determine-results'))

    #
    # WIN DETERMINATION RULES
    #
    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.win_totals << WinTotals(computer=MATCH.cw),
          Results(winner=MATCH.cc,
                  loser=MATCH.hc,
                  why=MATCH.explanation))
    def computer_wins(self, f1, f2, f3, win_totals, cw, explanation):
        print(f'\t\t computer_wins()')
        self.retract(f1) # determine-results
        self.retract(f2) # ComputerChoice
        self.retract(f3) # HumanChoice
        self.modify(win_totals, computer=cw + 1)
        print("Computer wins!", explanation)
        self.declare(Action('determine-play-again'))

    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.win_totals << WinTotals(human=MATCH.hw),
          Results(winner=MATCH.hc,
                  loser=MATCH.cc,
                  why=MATCH.explanation))
    def humans_wins(self, f1, f2, f3, win_totals, hw, explanation):
        self.retract(f1) # 'determine-results'
        self.retract(f2) # ComputerChoice
        self.retract(f3) # HumanChoice
        self.modify(win_totals, human=hw + 1)
        print("You win!", explanation)
        self.declare(Action('determine-play-again'))


    @Rule(AS.f1 << Action('determine-results'),
          AS.f2 << ComputerChoice(MATCH.cc),
          AS.f3 << HumanChoice(MATCH.hc),
          AS.ties_count << WinTotals(ties=MATCH.nt))
    def tie(self, f1, f2, f3, ties_count):
        self.retract(f1)
        self.retract(f2)
        self.retract(f3)
        self.modify(ties_count, ties=1)
        print("Tie! Ha-ha!")
        self.declare(Action('determine-play-again'))

    #
    # PLAY AGAIN RULE
    #
    @Rule(AS.f1 << Action('determine-play-again'),
          WinTotals(computer=MATCH.ct,
                    human=MATCH.ht,
                    ties=MATCH.tt))
    def play_again(self, f1, ct, ht, tt):
        self.retract(f1) # 'determine-play-again'
        if not self.yes_or_no("Play again?"):
            print("You won", ht, "game(s).")
            print("Computer won", ct, "game(s).")
            print("We tied", tt, "game(s).")
            self.halt()
        else:
            self.declare(Action('get-human-move'))

In [9]:
rps = RockPaperScissors()

In [14]:
rps.reset()
rps.run()

Lets play a game!
You choose rock, paper, or scissors,
and I'll do the same.
		 in get_human_move() --> declaring HumanChoice
		 in get-computer-move computer selectedrock
		 computer_wins()
Computer wins! Rock smashes scissors
You won 0 game(s).
Computer won 1 game(s).
We tied 0 game(s).
